# Setup

The name of this database is `ex_social`.

If you believe there is an error in the results, please post an issue to this repo.

In [1]:
%load_ext sql
%sql postgres://localhost/ex_social

'Connected: @ex_social'

## Questions

1. How many users are there in each house?

In [2]:
%%sql
SELECT
    count(user_id), house
FROM
    users
GROUP BY 
    house
ORDER BY 
    house
;

 * postgres://localhost/ex_social
4 rows affected.


count,house
4,Gryffindor
1,Hufflepuff
2,Ravenclaw
2,Slytherin


2. List all following links that were created before September 1st, 1993

In [3]:
%%sql
SELECT 
    *
FROM
    follows
WHERE
    date_created < timestamp '1993-09-01'
;

 * postgres://localhost/ex_social
6 rows affected.


user_id,follows,date_created
2,1,1989-01-01 00:00:00
3,1,1993-07-01 00:00:00
4,2,1988-08-08 00:00:00
4,1,1988-08-08 00:00:00
5,6,1986-01-10 00:00:00
7,1,1990-02-02 00:00:00


3. List all rows from the `follows` table, replacing both `user_id`s with first name. Hint: it may help to make this a VIEW

In [4]:
%%sql
CREATE VIEW named_followers AS
    SELECT
        u.first_name user_first,
        u.last_name user_last,
        u.house user_house,
        f.first_name follows_first,
        f.last_name follows_last,
        f.house follows_house,
        follows.*
    FROM
        follows
        JOIN
            users u
        ON
            u.user_id = follows.user_id
        JOIN

            users f
        ON
            f.user_id = follows.follows
;

 * postgres://localhost/ex_social
Done.


[]

In [5]:
%%sql
SELECT 
    user_first
    , follows_first
    , date_created
FROM
    named_followers
;

 * postgres://localhost/ex_social
20 rows affected.


user_first,follows_first,date_created
Harry,Ron,1993-09-01 00:00:00
Ron,Harry,1989-01-01 00:00:00
Hermonie,Harry,1993-07-01 00:00:00
Ron,Hermonie,1994-10-10 00:00:00
Hermonie,Ron,1995-03-01 00:00:00
Ginny,Ron,1988-08-08 00:00:00
Ginny,Harry,1988-08-08 00:00:00
Harry,Ginny,1994-04-02 00:00:00
Harry,Draco,2000-01-01 00:00:00
Draco,Harry,2000-01-02 00:00:00


4. List all the following links established before September 1st 1993, but this time use the users first names.

In [6]:
%%sql
SELECT 
    user_first, follows_first, date_created
FROM
    named_followers
WHERE
    date_created < timestamp '1993-09-01'

 * postgres://localhost/ex_social
6 rows affected.


user_first,follows_first,date_created
Ron,Harry,1989-01-01 00:00:00
Hermonie,Harry,1993-07-01 00:00:00
Ginny,Ron,1988-08-08 00:00:00
Ginny,Harry,1988-08-08 00:00:00
Draco,Tom,1986-01-10 00:00:00
Luna,Harry,1990-02-02 00:00:00


5. Give a count of how many people followed each user as of 1999-12-31. Give the result in term of "users full name, number of followers".

In [7]:
%%sql
SELECT
    u.first_name,
    u.last_name,
    count(f.follows) followers
FROM
        users u
    LEFT JOIN
        follows f
    ON 
        u.user_id = f.follows
    AND
        date_created < timestamp '1999-12-31'
GROUP BY 
    u.user_id, u.last_name, u.first_name
;

 * postgres://localhost/ex_social
9 rows affected.


first_name,last_name,followers
Ginny,Weasley,1
Tom,Riddle,1
Ron,Wesley,3
Cedric,Diggory,3
Hermonie,Granger,1
Draco,Malfoy,0
Luna,Lovegood,1
Harry,Potter,5
Cho,Chang,3


6. List the number of users each user follows

In [8]:
%%sql
SELECT
    u.first_name,
    u.last_name,
    count(f.user_id) num_following
FROM
        users u
    LEFT JOIN
        follows f
    ON 
        u.user_id = f.user_id
GROUP BY 
    f.user_id, u.last_name, u.first_name
;

 * postgres://localhost/ex_social
9 rows affected.


first_name,last_name,num_following
Draco,Malfoy,2
Harry,Potter,5
Hermonie,Granger,3
Ginny,Weasley,3
Ron,Wesley,2
Tom,Riddle,0
Luna,Lovegood,2
Cedric,Diggory,1
Cho,Chang,2


7. List all rows from `follows` where someone from one house follows someone from a different house.

In [9]:
%%sql
SELECT 
    user_first
    ,user_last
    ,user_house
    ,follows_first
    ,follows_last
    ,follows_house
FROM
    named_followers
WHERE
    user_house != follows_house
    

 * postgres://localhost/ex_social
10 rows affected.


user_first,user_last,user_house,follows_first,follows_last,follows_house
Harry,Potter,Gryffindor,Draco,Malfoy,Slytherin
Draco,Malfoy,Slytherin,Harry,Potter,Gryffindor
Luna,Lovegood,Ravenclaw,Harry,Potter,Gryffindor
Harry,Potter,Gryffindor,Luna,Lovegood,Ravenclaw
Harry,Potter,Gryffindor,Cho,Chang,Ravenclaw
Cho,Chang,Ravenclaw,Harry,Potter,Gryffindor
Cho,Chang,Ravenclaw,Cedric,Diggory,Hufflepuff
Cedric,Diggory,Hufflepuff,Cho,Chang,Ravenclaw
Hermonie,Granger,Gryffindor,Cedric,Diggory,Hufflepuff
Ginny,Weasley,Gryffindor,Cedric,Diggory,Hufflepuff


8. We define a _friendship_ as a relationship between two users where both follow each other. The friendship is established when the _later_ of the two links is established. 

In [10]:
%%sql
SELECT
    ff.user_id
    , ff.follows
    , greatest(fb.date_created, ff.date_created) 
        as friendship_established
FROM
    follows ff
    JOIN
    follows fb
    ON 
        ff.user_id = fb.follows
        AND
        fb.user_id = ff.follows
        AND
        ff.user_id < fb.user_id /* so we don't get any link twice */
;

 * postgres://localhost/ex_social
7 rows affected.


user_id,follows,friendship_established
1,2,1993-09-01 00:00:00
1,4,1994-04-02 00:00:00
1,5,2000-01-02 00:00:00
1,7,1996-10-01 00:00:00
1,8,1995-09-01 00:00:00
2,3,1995-03-01 00:00:00
8,9,1996-01-10 00:00:00


In [11]:
%%sql
SELECT
    ff.user_first
    , ff.user_last
    , ff.follows_first
    , ff.follows_last
    , greatest(fb.date_created, ff.date_created) 
        as friendship_established
FROM
    named_followers ff
    JOIN
    named_followers fb
    ON 
        ff.user_id = fb.follows
        AND
        fb.user_id = ff.follows
        AND
        ff.user_id < fb.user_id
;

 * postgres://localhost/ex_social
7 rows affected.


user_first,user_last,follows_first,follows_last,friendship_established
Harry,Potter,Ron,Wesley,1993-09-01 00:00:00
Ron,Wesley,Hermonie,Granger,1995-03-01 00:00:00
Harry,Potter,Ginny,Weasley,1994-04-02 00:00:00
Harry,Potter,Draco,Malfoy,2000-01-02 00:00:00
Harry,Potter,Luna,Lovegood,1996-10-01 00:00:00
Harry,Potter,Cho,Chang,1995-09-01 00:00:00
Cho,Chang,Cedric,Diggory,1996-01-10 00:00:00


9. List all unrequited followings (i.e. where A follows B but B does not follow A)

In [12]:
%%sql
SELECT
    f.user_id, f.follows
FROM
        follows f
    LEFT JOIN
        follows fb
    ON 
            f.follows = fb.user_id
        AND
            fb.follows = f.user_id
WHERE
    fb.follows IS NULL
;

 * postgres://localhost/ex_social
6 rows affected.


user_id,follows
3,1
4,2
5,6
7,8
3,9
4,9


In [13]:
%%sql
SELECT
    f.user_first
    , f.user_last
    , f.follows_first
    , f.follows_last
FROM
        named_followers f
    LEFT JOIN
        named_followers fb
    ON 
            f.follows = fb.user_id
        AND
            fb.follows = f.user_id
WHERE
    fb.follows IS NULL
;

 * postgres://localhost/ex_social
6 rows affected.


user_first,user_last,follows_first,follows_last
Hermonie,Granger,Harry,Potter
Ginny,Weasley,Ron,Wesley
Draco,Malfoy,Tom,Riddle
Luna,Lovegood,Cho,Chang
Hermonie,Granger,Cedric,Diggory
Ginny,Weasley,Cedric,Diggory
